In [90]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline

import glob
import os

import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics

In [91]:
 def loadMetricData(user, directory):
    path =  user + "/" + directory + "/"
#     print(path)
    all_files = glob.glob(os.path.join(path, "*.csv")) 
#     print(all_files)
    data = pd.DataFrame()
    list_ = []
    for file_ in all_files:
        data = pd.read_csv(file_,index_col=None, header=0)
        list_.append(data)
        data = pd.concat(list_)
    return data

In [92]:
# # clean times function
def cleanTime(df, col):
    df.col = pd.to_datetime(df.col)

In [93]:
# # replace NAN's
# def rmNAN(df, col):
#     df.col = df.col.fillna(0)

In [94]:
barbMetrics = loadMetricData('barbComp', "metrics")
barbMetrics.tail(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
1435,2016-05-02 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN
1436,2016-05-02 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN
1437,2016-05-02 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN
1438,2016-05-02 23:58:00,NaN,NaN,NaN,NaN,NaN,NaN
1439,2016-05-02 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
barbMetrics.dtypes

timestamp     object
heartrate    float64
steps        float64
calories     float64
gsr          float64
skintemp     float64
airtemp      float64
dtype: object

In [96]:
# clean metric data

barbMetrics.timestamp = pd.to_datetime(barbMetrics.timestamp)

barbMetrics.heartrate = barbMetrics.heartrate.fillna(0)
barbMetrics.steps = barbMetrics.steps.fillna(0)
barbMetrics.calories = barbMetrics.calories.fillna(0)
barbMetrics.gsr = barbMetrics.gsr.fillna(0)
barbMetrics.skintemp = barbMetrics.skintemp.fillna(0)
barbMetrics.airtemp = barbMetrics.airtemp.fillna(0)

barbMetrics.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
0,2016-04-26 00:00:00,0,0,0,0,0,0
1,2016-04-26 00:01:00,0,0,0,0,0,0
2,2016-04-26 00:02:00,0,0,0,0,0,0
3,2016-04-26 00:03:00,0,0,0,0,0,0
4,2016-04-26 00:04:00,0,0,0,0,0,0


In [97]:
barbActivities = loadMetricData('barbComp', "activities")
barbActivities.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,type,actual seconds,steps,calories,minutes,heart rate avg,heart rate min,heart rate max,state,version,id
0,2016-05-02 11:58:04,2016-05-02T11:58:04Z,America/New_York,-240,2016-05-02 12:02:15,2016-05-02T12:02:15Z,America/New_York,-240,walk,251,433,23.000000,4.183333,112.243027,NaN,NaN,complete,1,walk__2016-05-02T115804
1,2016-05-02 12:05:02,2016-05-02T12:05:02Z,America/New_York,-240,2016-05-02 12:17:42,2016-05-02T12:17:42Z,America/New_York,-240,walk,646,1152,71.300003,12.666667,114.549538,NaN,NaN,complete,1,walk__2016-05-02T120502
2,2016-05-02 13:01:23,2016-05-02T13:01:23Z,America/New_York,-240,2016-05-02 13:10:46,2016-05-02T13:10:46Z,America/New_York,-240,walk,307,485,18.600000,9.383333,102.285713,NaN,NaN,complete,1,walk__2016-05-02T130123
3,2016-05-02 16:13:28,2016-05-02T16:13:28Z,America/New_York,-240,2016-05-02 16:23:46,2016-05-02T16:23:46Z,America/New_York,-240,walk,272,486,31.100000,10.300000,121.375000,NaN,NaN,complete,1,walk__2016-05-02T161328
4,2016-05-02 16:47:41,2016-05-02T16:47:41Z,America/New_York,-240,2016-05-02 16:53:08,2016-05-02T16:53:08Z,America/New_York,-240,walk,233,435,26.299999,5.450000,116.309013,NaN,NaN,complete,1,walk__2016-05-02T164741


In [98]:
barbSleep = loadMetricData('barbComp', "sleep")
barbSleep.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,light mins,deep mins,...,toss turns,type,actual seconds,calories,heart rate avg,heart rate min,heart rate max,state,version,id
0,2016-04-29 22:38:22,2016-04-29T22:38:22Z,America/New_York,-240,2016-04-29 23:11:22,2016-04-29T23:11:22Z,America/New_York,-240,32,0,...,0,sleep,1980,41.599998,77.544754,NaN,NaN,complete,2,sleep__2016-04-29T223822
1,2016-04-30 04:54:42,2016-04-30T04:54:42Z,America/New_York,-240,2016-04-30 13:54:42,2016-04-30T13:54:42Z,America/New_York,-240,275,139,...,27,sleep,32400,622.700012,67.379333,NaN,NaN,complete,2,sleep__2016-04-30T045442
0,2016-05-01 17:35:39,2016-05-01T17:35:39Z,America/New_York,-240,2016-05-01 18:13:39,2016-05-01T18:13:39Z,America/New_York,-240,37,0,...,2,sleep,2280,43.700001,59.494274,NaN,NaN,complete,2,sleep__2016-05-01T173539
1,2016-05-02 01:57:39,2016-05-02T01:57:39Z,America/New_York,-240,2016-05-02 03:09:39,2016-05-02T03:09:39Z,America/New_York,-240,71,0,...,4,sleep,4320,89.099998,79.341530,NaN,NaN,complete,2,sleep__2016-05-02T015739
2,2016-05-02 03:58:39,2016-05-02T03:58:39Z,America/New_York,-240,2016-05-02 10:40:39,2016-05-02T10:40:39Z,America/New_York,-240,210,103,...,18,sleep,23760,460.600006,58.690727,NaN,NaN,complete,2,sleep__2016-05-02T035839


In [99]:
barbSleep.dtypes

start time              object
start time ISO          object
start time timezone     object
start time offset      float64
end time                object
end time ISO            object
end time timezone       object
end time offset        float64
light mins             float64
deep mins              float64
rem mins               float64
interruption mins      float64
unknown mins           float64
interruptions          float64
toss turns             float64
type                    object
actual seconds         float64
calories               float64
heart rate avg         float64
heart rate min         float64
heart rate max         float64
state                   object
version                float64
id                      object
dtype: object

In [100]:
barbJournal = loadMetricData('barbComp', "journal")
barbJournal.tail(5)

,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,happy,calm,anxious,sad,angry,hungry,tired,bored
113,5/2/16 7:15,5/2/16 8:15,5/2/16,7:15 AM,8:15 AM,Train Commute,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN
114,5/2/16 8:15,5/2/16 9:30,5/2/16,8:15 AM,9:30 AM,setting up computers,work,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN
115,5/2/16 9:30,5/2/16 11:13,5/2/16,9:30 AM,11:13 AM,Class and setting up computers,work,NaN,NaN,NaN,NaN,NaN,6,6,NaN,NaN,NaN,NaN
116,5/2/16 11:13,5/2/16 12:00,5/2/16,11:13 AM,12:00 PM,Work,work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,5/2/16 12:00,5/2/16 13:18,5/2/16,12:00 PM,1:18 PM,meeting with anne,work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
barbJournal.dtypes

starttime       object
endtime         object
date            object
start           object
end             object
activity        object
location        object
generalemo      object
qualitative    float64
excited        float64
happy          float64
calm           float64
anxious        float64
sad            float64
angry          float64
hungry         float64
tired          float64
bored          float64
dtype: object

In [102]:
# data= barbJournal.iloc[:0]
# clean metric data
barbJournal["starttime"]= pd.to_datetime(barbJournal.starttime)

# data= barbJournal.iloc[:1]
barbJournal["endtime"]= pd.to_datetime(barbJournal.endtime)


# sarahJournal.qualitative = sarahJournal.qualitative.convert_objects(convert_numeric=True)
# sarahJournal.excited = sarahJournal.excited.convert_objects(convert_numeric=True)
# sarahJournal.happy = sarahJournal.happy.convert_objects(convert_numeric=True)

barbJournal.qualitative = barbJournal.qualitative.fillna(0)
barbJournal.excited = barbJournal.excited.fillna(0)
barbJournal.happy = barbJournal.happy.fillna(0)
barbJournal.calm = barbJournal.calm.fillna(0)
barbJournal.anxious = barbJournal.anxious.fillna(0)
barbJournal.sad = barbJournal.sad.fillna(0)
barbJournal.angry = barbJournal.angry.fillna(0)
barbJournal.hungry = barbJournal.hungry.fillna(0)
barbJournal.tired = barbJournal.tired.fillna(0)
barbJournal.bored = barbJournal.bored.fillna(0)

print(barbJournal.shape)
barbJournal.head(5)
# barbJournal.tail(5)

(118, 18)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,happy,calm,anxious,sad,angry,hungry,tired,bored
0,2016-04-26 09:28:00,2016-04-26 09:33:00,4/26/16,9:28 AM,9:33 AM,"setting up basis, drinking coffee, listening t...",work,Calm,0,1,2,6,0,0,0,0,2,1
1,2016-04-26 09:33:00,2016-04-26 10:02:00,4/26/16,9:33 AM,10:02 AM,"listening to 80's pop, going through work email",work,Calm,0,0,2,6,0,0,0,0,2,0
2,2016-04-26 10:02:00,2016-04-26 10:25:00,4/26/16,10:02 AM,10:25 AM,meeting with student,work,Calm,0,0,0,6,0,0,0,0,4,0
3,2016-04-26 10:25:00,2016-04-26 10:35:00,4/26/16,10:25 AM,10:35 AM,meeting with mick,work,Calm,0,0,0,6,0,0,0,0,4,0
4,2016-04-26 10:35:00,2016-04-26 10:45:00,4/26/16,10:35 AM,10:45 AM,meeting with student,school,Calm,0,0,0,6,0,0,0,0,4,0


In [103]:

barbJournal['boolexcited'] =  barbJournal['excited'] > 0
barbJournal['boolhappy'] =  barbJournal['happy'] > 0
barbJournal['boolcalm'] =  barbJournal['calm'] > 0
barbJournal['boolanxious'] =  barbJournal['anxious'] > 0
barbJournal['boolsad'] =  barbJournal['sad'] > 0
barbJournal['boolangry'] =  barbJournal['angry'] > 0
barbJournal['booltired'] =  barbJournal['tired'] > 0
barbJournal['boolhungry'] =  barbJournal['hungry'] > 0
barbJournal['boolbored'] =  barbJournal['bored'] > 0


for i in range(0, barbJournal.shape[0]):
#     barbJournal['intqual'] = int(barbJournal.iloc[i, 8])
    barbJournal['intexcited'] = int(barbJournal.iloc[i, 9])
    barbJournal['inthappy'] = int(barbJournal.iloc[i, 10]) 
    barbJournal['intcalm'] = int(barbJournal.iloc[i, 11])
    barbJournal['intanxious'] = int(barbJournal.iloc[i, 12])
    barbJournal['intsad'] = int(barbJournal.iloc[i, 13]) 
    barbJournal['intangry'] = int(barbJournal.iloc[i, 14]) 
    barbJournal['inttired'] = int(barbJournal.iloc[i, 15]) 
    barbJournal['inthungry'] = int(barbJournal.iloc[i, 16]) 
    barbJournal['intbored'] = int(barbJournal.iloc[i, 17]) 
    
barbJournal['exercise'] = barbJournal['activity'].str.contains('exercise')
barbJournal['code'] = barbJournal['activity'].str.contains('code')
barbJournal['anne'] = barbJournal['activity'].str.contains('anne')
barbJournal['will'] = barbJournal['activity'].str.contains('will')
barbJournal['mick'] = barbJournal['activity'].str.contains('mick')
barbJournal['geyer'] = barbJournal['activity'].str.contains('geyer')

barbJournal['home'] = barbJournal['location'].str.contains('home')
barbJournal['work'] = barbJournal['location'].str.contains("work")
barbJournal['school'] = barbJournal['location'].str.contains("school")




boolexcited = barbJournal.iloc[:, 18].values.reshape(barbJournal.shape[0], 1)
boolhappy = barbJournal.iloc[:, 19].values.reshape(barbJournal.shape[0], 1)
boolcalm = barbJournal.iloc[:, 20].values.reshape(barbJournal.shape[0], 1)
boolanxious = barbJournal.iloc[:, 21].values.reshape(barbJournal.shape[0], 1)
boolsad = barbJournal.iloc[:, 22].values.reshape(barbJournal.shape[0], 1)
boolangry = barbJournal.iloc[:, 23].values.reshape(barbJournal.shape[0], 1)
booltired = barbJournal.iloc[:, 24].values.reshape(barbJournal.shape[0], 1)
boolhungry = barbJournal.iloc[:, 25].values.reshape(barbJournal.shape[0], 1)
boolbored = barbJournal.iloc[:, 26].values.reshape(barbJournal.shape[0], 1)
intexcited = barbJournal.iloc[:, 27].values.reshape(barbJournal.shape[0], 1)
inthappy = barbJournal.iloc[:, 28].values.reshape(barbJournal.shape[0], 1)
intcalm = barbJournal.iloc[:, 29].values.reshape(barbJournal.shape[0], 1)
intanxious = barbJournal.iloc[:, 30].values.reshape(barbJournal.shape[0], 1)
intsad = barbJournal.iloc[:, 31].values.reshape(barbJournal.shape[0], 1)
intangry = barbJournal.iloc[:, 32].values.reshape(barbJournal.shape[0], 1)
inttired = barbJournal.iloc[:, 33].values.reshape(barbJournal.shape[0], 1)
inthungry = barbJournal.iloc[:, 34].values.reshape(barbJournal.shape[0], 1)
intbored = barbJournal.iloc[:, 35].values.reshape(barbJournal.shape[0], 1)
exercise = barbJournal.iloc[:, 36].values.reshape(barbJournal.shape[0], 1)
code = barbJournal.iloc[:, 37].values.reshape(barbJournal.shape[0], 1)
anne = barbJournal.iloc[:, 38].values.reshape(barbJournal.shape[0], 1)
will = barbJournal.iloc[:, 39].values.reshape(barbJournal.shape[0], 1)
mick = barbJournal.iloc[:, 40].values.reshape(barbJournal.shape[0], 1)
geyer = barbJournal.iloc[:, 41].values.reshape(barbJournal.shape[0], 1)
home = barbJournal.iloc[:, 42].values.reshape(barbJournal.shape[0], 1)
work = barbJournal.iloc[:, 43].values.reshape(barbJournal.shape[0], 1)
school = barbJournal.iloc[:, 44].values.reshape(barbJournal.shape[0], 1)

barbJournal.head(5)

,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,intbored,exercise,code,anne,will,mick,geyer,home,work,school
0,2016-04-26 09:28:00,2016-04-26 09:33:00,4/26/16,9:28 AM,9:33 AM,"setting up basis, drinking coffee, listening t...",work,Calm,0,1,...,0,False,False,False,False,False,False,False,True,False
1,2016-04-26 09:33:00,2016-04-26 10:02:00,4/26/16,9:33 AM,10:02 AM,"listening to 80's pop, going through work email",work,Calm,0,0,...,0,False,False,False,False,False,False,False,True,False
2,2016-04-26 10:02:00,2016-04-26 10:25:00,4/26/16,10:02 AM,10:25 AM,meeting with student,work,Calm,0,0,...,0,False,False,False,False,False,False,False,True,False
3,2016-04-26 10:25:00,2016-04-26 10:35:00,4/26/16,10:25 AM,10:35 AM,meeting with mick,work,Calm,0,0,...,0,False,False,False,False,True,False,False,True,False
4,2016-04-26 10:35:00,2016-04-26 10:45:00,4/26/16,10:35 AM,10:45 AM,meeting with student,school,Calm,0,0,...,0,False,False,False,False,False,False,False,False,True


In [104]:
# barbJournal.dtypes

In [105]:
# barbJournal.excited.convert_objects(convert_numeric=True)

In [106]:
# function that takes any journal or activity data and creates indicators for activities or emotions

def makeIndicators(newvar, metricData, journalData, journal_startdatetime_loc, journal_enddatetime_loc, journal_activity_loc):
    '''
    Create boolean indicator variable for activities or emotions
    newvar, string for name of new indicator variable
    activity, string that matches the name of the activity or emotion in the journalData data frame
    bioData, data frame of input biometric data
    journalData, data frame of input journal or activity data
    journal_startdatetime_loc, int of position of the start datetime in journalData
    journal_enddatetime_loc, int of position of the end datetime in journalData
    journal_activity_loc, int of position of the activity or emotion in journalData
    '''

    metricData[newvar] = False

    for i in range(0, journalData.shape[0]):
        metricData['xlt'] = (metricData['timestamp'] >= pd.to_datetime(journalData.iloc[i, journal_startdatetime_loc]))
        metricData['xgt'] = (metricData['timestamp'] <= pd.to_datetime(journalData.iloc[i, journal_enddatetime_loc]))
#         metricData['xw'] = (journalData.iloc[i, journal_activity_loc])
        metricData['xw'] = (journalData.iloc[i, journal_activity_loc] == True)
        metricData['xsum'] = metricData['xlt'].astype(int) + metricData['xgt'].astype(int) + metricData['xw'].astype(int)
        metricData.loc[metricData['xsum']==3, newvar] = True
    
    metricData = metricData.drop(['xlt', 'xgt', 'xw', 'xsum'], axis=1)
    return metricData

In [107]:
# indicator for walking
totalData = makeIndicators('excitement', barbMetrics, barbJournal, 0, 1, 18)
totalData = makeIndicators('happy', barbMetrics, barbJournal, 0, 1, 19)
totalData = makeIndicators('calm', barbMetrics, barbJournal, 0, 1, 20)
totalData = makeIndicators('anxious', barbMetrics, barbJournal, 0, 1, 21)
totalData = makeIndicators('sad', barbMetrics, barbJournal, 0, 1, 22)
totalData = makeIndicators('angry', barbMetrics, barbJournal, 0, 1, 23)
totalData = makeIndicators('hungry', barbMetrics, barbJournal, 0, 1, 24)
totalData = makeIndicators('tired', barbMetrics, barbJournal, 0, 1, 25)
totalData = makeIndicators('bored', barbMetrics, barbJournal, 0, 1, 26)

totalData = makeIndicators('exercise', barbMetrics, barbJournal, 0, 1, 36)
totalData = makeIndicators('code', barbMetrics, barbJournal, 0, 1, 37)
totalData = makeIndicators('anne', barbMetrics, barbJournal, 0, 1, 38)
totalData = makeIndicators('will', barbMetrics, barbJournal, 0, 1, 39)
totalData = makeIndicators('mick', barbMetrics, barbJournal, 0, 1, 40)
totalData = makeIndicators('geyer', barbMetrics, barbJournal, 0, 1, 41)
totalData = makeIndicators('home', barbMetrics, barbJournal, 0, 1, 42)
totalData = makeIndicators('work', barbMetrics, barbJournal, 0, 1, 43)
totalData = makeIndicators('school', barbMetrics, barbJournal, 0, 1, 44)


In [108]:
print(totalData.describe())

          heartrate         steps      calories           gsr      skintemp  \
count  10080.000000  10080.000000  10080.000000  10080.000000  10080.000000   
mean      65.284623      3.466567      1.658363      0.001082     70.396171   
std       35.596657     16.000542      1.467958      0.018148     35.013143   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%       57.000000      0.000000      1.100000      0.000055     82.400000   
50%       76.000000      0.000000      1.300000      0.000060     86.900000   
75%       88.000000      0.000000      1.900000      0.000073     88.700000   
max      198.000000    124.000000     17.200000      0.719000     97.700000   

            airtemp   excitement      happy      calm   anxious    ...      \
count  10080.000000        10080      10080     10080     10080    ...       
mean      67.822421  0.000595238  0.0570437  0.500198  0.338294    ...       
std       33.805046    0.0243914   0.231938  0.500025 

In [109]:
print(totalData.head())

            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-04-26 00:00:00          0      0         0    0         0        0   
1 2016-04-26 00:01:00          0      0         0    0         0        0   
2 2016-04-26 00:02:00          0      0         0    0         0        0   
3 2016-04-26 00:03:00          0      0         0    0         0        0   
4 2016-04-26 00:04:00          0      0         0    0         0        0   

  excitement  happy   calm  ...    bored exercise   code   anne   will   mick  \
0      False  False  False  ...    False    False  False  False  False  False   
1      False  False  False  ...    False    False  False  False  False  False   
2      False  False  False  ...    False    False  False  False  False  False   
3      False  False  False  ...    False    False  False  False  False  False   
4      False  False  False  ...    False    False  False  False  False  False   

   geyer   home   work school  
0  False  False  F

In [110]:
totalData.to_csv('barbTotal.csv')

In [111]:
totalData.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp,excitement,happy,calm,...,bored,exercise,code,anne,will,mick,geyer,home,work,school
0,2016-04-26 00:00:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2016-04-26 00:01:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2016-04-26 00:02:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2016-04-26 00:03:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2016-04-26 00:04:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [112]:
# find day of week (0-6, Mon-Sun)
totalData['weekday'] = totalData['timestamp'].dt.weekday
totalData['month'] = totalData['timestamp'].dt.month
totalData['hour'] = totalData['timestamp'].dt.hour
totalData['min'] = totalData['timestamp'].dt.minute


print(totalData.shape)
print(totalData.head(5))

(10080, 29)
            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-04-26 00:00:00          0      0         0    0         0        0   
1 2016-04-26 00:01:00          0      0         0    0         0        0   
2 2016-04-26 00:02:00          0      0         0    0         0        0   
3 2016-04-26 00:03:00          0      0         0    0         0        0   
4 2016-04-26 00:04:00          0      0         0    0         0        0   

  excitement  happy   calm ...   will   mick  geyer   home   work school  \
0      False  False  False ...  False  False  False  False  False  False   
1      False  False  False ...  False  False  False  False  False  False   
2      False  False  False ...  False  False  False  False  False  False   
3      False  False  False ...  False  False  False  False  False  False   
4      False  False  False ...  False  False  False  False  False  False   

  weekday month hour min  
0       1     4    0   0  
1       1     

In [113]:
heartrate = totalData.iloc[:, 1].values.reshape(totalData.shape[0], 1)
steps = totalData.iloc[:, 2].values.reshape(totalData.shape[0], 1)
calories = totalData.iloc[:, 3].values.reshape(totalData.shape[0], 1)
gsr = totalData.iloc[:, 4].values.reshape(totalData.shape[0], 1)
skintemp = totalData.iloc[:, 5].values.reshape(totalData.shape[0], 1)
airtemp = totalData.iloc[:, 6].values.reshape(totalData.shape[0], 1)
# activity = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
exercise = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
code = totalData.iloc[:, 18].values.reshape(totalData.shape[0], 1)
anne = totalData.iloc[:, 19].values.reshape(totalData.shape[0], 1)
will = totalData.iloc[:, 20].values.reshape(totalData.shape[0], 1)
mick = totalData.iloc[:, 21].values.reshape(totalData.shape[0], 1)
geyer = totalData.iloc[:, 22].values.reshape(totalData.shape[0], 1)
home = totalData.iloc[:, 23].values.reshape(totalData.shape[0], 1)
work = totalData.iloc[:, 24].values.reshape(totalData.shape[0], 1)
school = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# DayOfWeek = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# Month = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
weekday = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
# month = totalData.iloc[:, 29].values.reshape(totalData.shape[0], 1)
hour = totalData.iloc[:, 27].values.reshape(totalData.shape[0], 1)
curmin = totalData.iloc[:, 28].values.reshape(totalData.shape[0], 1)

In [114]:
# concatenate to numpy dataset
X = np.concatenate((
        heartrate, 
        steps, 
        calories, 
        gsr, 
        skintemp, 
        airtemp, 
#         exercise, 
#         code, 
#         anne, 
#         will, 
#         mick, 
#         geyer, 
#         home, 
#         work, 
#         school,
        weekday, 
        hour, 
        curmin
    ), axis=1)

X.shape

(10080, 9)

In [115]:
# define y
y = totalData.excitement.values
y.shape

(10080,)

In [116]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [117]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [118]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

# Excited Model Results

In [119]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000708616780045 0.000330687830688
 Accuracy Rate     0.99929138322    0.999669312169
True Positives               0.0               0.0
True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)

In [120]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set        Test Set    
------------------- ----------------- -----------------
         Error Rate 0.000850340136054 0.000661375661376
      Accuracy Rate    0.999149659864    0.999338624339
     True Positives               0.0               0.0
     True Negatives                --    0.999669202779


(0.00066137566137566134, 0.9993386243386243, 0.0, 0.99966920277869664)

In [121]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00283446712018 0.00297619047619
 Accuracy Rate    0.99716553288   0.997023809524
True Positives              0.0              0.0
True Negatives               --    0.99735362223


(0.002976190476190476, 0.99702380952380953, 0.0, 0.99735362222957324)

In [122]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000708616780045 0.000330687830688
 Accuracy Rate     0.99929138322    0.999669312169
True Positives               0.0               0.0
True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)

In [123]:
# define y
y = totalData.happy.values
y.shape

(10080,)

In [124]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [125]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Happy Model Results

In [126]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0605158730159 0.0545634920635
 Accuracy Rate  0.939484126984  0.945436507937
True Positives 0.0144927536232 0.0248447204969
True Negatives              --  0.997205728257


(0.054563492063492064,
 0.94543650793650791,
 0.024844720496894408,
 0.99720572825707299)

In [127]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0670351473923 0.0648148148148
      Accuracy Rate  0.932964852608  0.935185185185
     True Positives 0.0821256038647 0.0683229813665
     True Negatives              --  0.983932937478


(0.064814814814814811,
 0.93518518518518523,
 0.068322981366459631,
 0.98393293747816979)

In [128]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0789399092971 0.0770502645503
 Accuracy Rate  0.921060090703   0.92294973545
True Positives  0.135265700483  0.111801242236
True Negatives              --  0.968564442892


(0.077050264550264549,
 0.92294973544973546,
 0.11180124223602485,
 0.96856444289207122)

In [129]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0573979591837 0.0512566137566
 Accuracy Rate  0.942602040816  0.948743386243
True Positives 0.0265700483092 0.0372670807453
True Negatives              --             1.0


(0.051256613756613757, 0.94874338624338628, 0.037267080745341616, 1.0)

In [130]:
# define y
y = totalData.calm.values
y.shape

(10080,)

In [131]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [132]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Calm Model Results

In [133]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate  0.43324829932 0.444775132275
 Accuracy Rate  0.56675170068 0.555224867725
True Positives 0.141313619562 0.133114754098
True Negatives             -- 0.984656437625


(0.44477513227513227,
 0.55522486772486768,
 0.13311475409836065,
 0.98465643762508337)

In [134]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.298894557823 0.298941798942
      Accuracy Rate 0.701105442177 0.701058201058
     True Positives 0.683821438726 0.685901639344
     True Negatives             -- 0.716477651768


(0.29894179894179895,
 0.70105820105820105,
 0.68590163934426229,
 0.71647765176784528)

In [135]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.336451247166  0.33630952381
 Accuracy Rate 0.663548752834  0.66369047619
True Positives 0.877452374183 0.875409836066
True Negatives             -- 0.448298865911


(0.33630952380952384,
 0.66369047619047616,
 0.87540983606557377,
 0.44829886591060708)

In [136]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.321145124717 0.315806878307
 Accuracy Rate 0.678854875283 0.684193121693
True Positives 0.963036678988 0.965245901639
True Negatives             --  0.39826551034


(0.31580687830687831,
 0.68419312169312163,
 0.96524590163934432,
 0.39826551034022684)

In [137]:
# define y
y = totalData.anxious.values
y.shape

(10080,)

In [138]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [139]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Anxious Model Results

In [140]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.271258503401 0.264880952381
 Accuracy Rate 0.728741496599 0.735119047619
True Positives  0.27870913663    0.294921875
True Negatives             --         0.9605


(0.26488095238095238, 0.73511904761904767, 0.294921875, 0.96050000000000002)

In [141]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.328798185941 0.325066137566
      Accuracy Rate 0.671201814059 0.674933862434
     True Positives 0.402347024308    0.435546875
     True Negatives             --         0.7975


(0.32506613756613756, 0.67493386243386244, 0.435546875, 0.79749999999999999)

In [142]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.371315192744 0.376322751323
 Accuracy Rate 0.628684807256 0.623677248677
True Positives 0.108130762783   0.1279296875
True Negatives             --         0.8775


(0.37632275132275134, 0.62367724867724861, 0.1279296875, 0.87749999999999995)

In [143]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.428287981859 0.428902116402
 Accuracy Rate 0.571712018141 0.571097883598
True Positives 0.906119027661     0.91015625
True Negatives             --         0.3975


(0.4289021164021164, 0.5710978835978836, 0.91015625, 0.39750000000000002)

In [144]:
# define y
y = totalData.sad.values
y.shape

(10080,)

In [145]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [146]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Sad Model Values

In [147]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0816326530612 0.0839947089947
 Accuracy Rate  0.918367346939  0.916005291005
True Positives  0.142553191489  0.146464646465
True Negatives              --  0.969922151451


(0.083994708994708997,
 0.91600529100529104,
 0.14646464646464646,
 0.96992215145081384)

In [148]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0676020408163 0.0664682539683
      Accuracy Rate  0.932397959184  0.933531746032
     True Positives             0.0             0.0
     True Negatives              --  0.998938428875


(0.066468253968253968, 0.93353174603174605, 0.0, 0.99893842887473461)

In [149]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0735544217687  0.0701058201058
 Accuracy Rate  0.926445578231   0.929894179894
True Positives 0.0234042553191 0.00505050505051
True Negatives              --   0.994692144374


(0.070105820105820102,
 0.92989417989417988,
 0.0050505050505050509,
 0.99469214437367304)

In [150]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0668934240363 0.0658068783069
 Accuracy Rate  0.933106575964  0.934193121693
True Positives             0.0             0.0
True Negatives              --  0.999646142958


(0.065806878306878314, 0.93419312169312163, 0.0, 0.99964614295824483)

In [151]:
# define y
y = totalData.angry.values
y.shape

(10080,)

In [152]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [153]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Angry Model Results

In [154]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.208475056689 0.199074074074
 Accuracy Rate 0.791524943311 0.800925925926
True Positives 0.298369565217 0.330729166667
True Negatives             -- 0.960992907801


(0.19907407407407407,
 0.80092592592592593,
 0.33072916666666669,
 0.96099290780141844)

In [155]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.357568027211 0.346891534392
      Accuracy Rate 0.642431972789 0.653108465608
     True Positives 0.845652173913      0.8515625
     True Negatives             --  0.58554964539


(0.34689153439153442, 0.65310846560846558, 0.8515625, 0.58554964539007093)

In [156]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.251842403628 0.243055555556
 Accuracy Rate 0.748157596372 0.756944444444
True Positives 0.157065217391 0.170572916667
True Negatives             -- 0.956560283688


(0.24305555555555555,
 0.75694444444444442,
 0.17057291666666666,
 0.95656028368794321)

In [157]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate   0.262755102041   0.255291005291
 Accuracy Rate   0.737244897959   0.744708994709
True Positives 0.00380434782609 0.00260416666667
True Negatives               --   0.997340425532


(0.25529100529100529,
 0.74470899470899465,
 0.0026041666666666665,
 0.99734042553191493)

In [158]:
# define y
y = totalData.hungry.values
y.shape

(10080,)

In [159]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [160]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Hungry Model Results

In [161]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0493197278912 0.0416666666667
 Accuracy Rate  0.950680272109  0.958333333333
True Positives             0.0             0.0
True Negatives              --             1.0


(0.041666666666666664, 0.95833333333333337, 0.0, 1.0)

In [162]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0514455782313 0.0433201058201
      Accuracy Rate  0.948554421769   0.95667989418
     True Positives             0.0             0.0
     True Negatives              --  0.998274672188


(0.043320105820105821, 0.95667989417989419, 0.0, 0.99827467218771571)

In [163]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0639172335601 0.0575396825397
 Accuracy Rate   0.93608276644   0.94246031746
True Positives 0.0316091954023 0.0238095238095
True Negatives              --  0.982401656315


(0.057539682539682536,
 0.94246031746031744,
 0.023809523809523808,
 0.98240165631469978)

In [164]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0493197278912 0.0416666666667
 Accuracy Rate  0.950680272109  0.958333333333
True Positives             0.0             0.0
True Negatives              --             1.0


(0.041666666666666664, 0.95833333333333337, 0.0, 1.0)

In [165]:
# define y
y = totalData.tired.values
y.shape

(10080,)

In [166]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [167]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Tired Model Results

In [168]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00127551020408 0.000992063492063
 Accuracy Rate   0.998724489796    0.999007936508
True Positives              0.0               0.0
True Negatives               --               1.0


(0.00099206349206349201, 0.99900793650793651, 0.0, 1.0)

In [169]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set        Test Set    
------------------- ---------------- -----------------
         Error Rate 0.00127551020408 0.000992063492063
      Accuracy Rate   0.998724489796    0.999007936508
     True Positives              0.0               0.0
     True Negatives               --               1.0


(0.00099206349206349201, 0.99900793650793651, 0.0, 1.0)

In [170]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00297619047619 0.00297619047619
 Accuracy Rate   0.997023809524   0.997023809524
True Positives              0.0              0.0
True Negatives               --   0.998013902681


(0.002976190476190476, 0.99702380952380953, 0.0, 0.99801390268123136)

In [171]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00127551020408 0.000992063492063
 Accuracy Rate   0.998724489796    0.999007936508
True Positives              0.0               0.0
True Negatives               --               1.0


(0.00099206349206349201, 0.99900793650793651, 0.0, 1.0)

In [172]:
# define y
y = totalData.bored.values
y.shape

(10080,)

In [173]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [174]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Bored Model Results

In [175]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000708616780045 0.000330687830688
 Accuracy Rate     0.99929138322    0.999669312169
True Positives               0.0               0.0
True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)

In [176]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set        Test Set    
------------------- ----------------- -----------------
         Error Rate 0.000708616780045 0.000330687830688
      Accuracy Rate     0.99929138322    0.999669312169
     True Positives               0.0               0.0
     True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)

In [177]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00283446712018 0.00297619047619
 Accuracy Rate    0.99716553288   0.997023809524
True Positives              0.0              0.0
True Negatives               --    0.99735362223


(0.002976190476190476, 0.99702380952380953, 0.0, 0.99735362222957324)

In [178]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000708616780045 0.000330687830688
 Accuracy Rate     0.99929138322    0.999669312169
True Positives               0.0               0.0
True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)